In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

#!pip install imbalanced-learn

In [2]:
#Colocamos el lugar de donde extraer el csv
data_location= 'https://raw.githubusercontent.com/mwolinsky/Ranking_predictor/main/england-premier-league-players-2018-to-2019-stats.csv'

In [3]:
#Leemos el csv con lo jugadores como indice
df= pd.read_csv(data_location)

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

#!pip install imbalanced-learn

#Colocamos el lugar de donde extraer el csv
data_location= 'https://raw.githubusercontent.com/mwolinsky/Ranking_predictor/main/england-premier-league-players-2018-to-2019-stats.csv'

#Leemos el csv con lo jugadores como indice
df= pd.read_csv(data_location)

In [39]:
df_def=df[df.position=="Defender"]

In [40]:
df_def["min_per_total_match"]=df_def.minutes_played_overall/(38*90)

<ipython-input-40-3287282ed513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def["min_per_total_match"]=df_def.minutes_played_overall/(38*90)


In [41]:
df_def= df_def.replace(-1,99999) 


In [42]:
df_def['top_rank']=df_def.loc[:,['rank_in_league_top_attackers','rank_in_league_top_midfielders','rank_in_league_top_defenders']].apply(lambda x: x.min(),axis=1)


In [43]:
categorical_columns=['position','Current Club','nationality']
for column in categorical_columns:
    dummies = pd.get_dummies(df_def[column], prefix=column,drop_first=True)
    df_def = pd.concat([df_def, dummies], axis=1)
    df_def = df_def.drop(columns=column)

In [44]:
df_def['top_def_15']=df.rank_in_league_top_defenders.apply(lambda x: 1 if x>0 and x<=15 else 0)
df_def['top_15']= df_def.apply(lambda x: 1 if x.top_def_15==1 else 0,axis=1)

In [45]:
X=df.loc[:,[min_per_conceded_overall,conceded_per_90_overall,minutes_played_overall,"goals_overall","clean_sheets_away"]]
y= df.top_15
from sklearn.model_selection import train_test_split

#Con estratificación en y

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=162)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)


In [46]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
model_xg = XGBClassifier(n_jobs=-1, use_label_encoder=False)

cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

#Hacemos un Grid Search para optimizar hiperparámetros
from sklearn.model_selection import RandomizedSearchCV
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

clf = RandomizedSearchCV(model_xg, param_distributions=params, cv=cv, verbose=1, n_jobs=-1,random_state=2123,)

clf.fit(X_train,y_train)

print(clf.best_params_)

cv_score = clf.best_score_
test_score = clf.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[19:12:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'subsample': 0.5, 'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.3, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.4}
Cross-validation score: 1.0
Test score: 1.0


In [47]:
#Entrenamos el modelo con los mejores hiperparámetros
xgb = XGBClassifier(subsample= 0.8999999999999999,n_estimators=100, max_depth=6, learning_rate= 0.1,colsample_bytree= 0.6, colsample_bylevel= 0.7999999999999999)
xgb.fit(X_train, y_train)
#Testeamos el modelo con los hiperparámetros tuneados
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
xgb_auc = roc_auc_score(y_test, xgb.predict(X_test))

print("El valor del AUC es: ", xgb_auc)
print('El accuracy es:', accuracy_score(y_test, xgb.predict(X_test)))




[19:12:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
El valor del AUC es:  0.9888888888888888
El accuracy es: 0.9791666666666666


C:\Users\Maty\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [48]:
importance_xgb=pd.DataFrame({'features': X.columns,'importance':xgb.feature_importances_}).sort_values(by='importance',ascending=False)

importance_xgb.head(10)

,features,importance
2,minutes_played_overall,0.174274
31,min_per_conceded_overall,0.154645
49,Current Club_Liverpool,0.133427
4,minutes_played_away,0.124942
22,yellow_cards_overall,0.107111
50,Current Club_Manchester City,0.059730
16,clean_sheets_overall,0.053742
17,clean_sheets_home,0.053060
30,conceded_per_90_overall,0.039841
27,goals_per_90_home,0.017600


In [83]:
df_def['min_per_conceded_overall']= df.min_per_conceded_overall/38
df_def['minutes_played_overall']= df.minutes_played_overall/38
df_def['goals_overall']= df.goals_overall/38
df_def["clean_sheets_away"]=df_def.clean_sheets_away/19

df_def["goals_involved_per_90_overall"]=df_def.goals_involved_per_90_overall/38

In [84]:
smote = SMOTE(random_state = 11)
X_train, y_train = smote.fit_resample(X_train, y_train)

from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
model_xg = XGBClassifier(n_jobs=-1, use_label_encoder=False)

cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

#Hacemos un Grid Search para optimizar hiperparámetros
from sklearn.model_selection import RandomizedSearchCV
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

clf = RandomizedSearchCV(model_xg, param_distributions=params, cv=cv, verbose=1, n_jobs=-1,random_state=2123,)

clf.fit(X_train,y_train)

print(clf.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[19:42:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'subsample': 0.5, 'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.3, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.4}


In [85]:
xgb = XGBClassifier(subsample= 0.5,n_estimators=100, max_depth=15, learning_rate= 0.3,colsample_bytree= 0.8999, colsample_bylevel= 0.4)
xgb.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
xgb_auc = roc_auc_score(y_test, xgb.predict(X_test))

print("El valor del AUC es: ", xgb_auc)
print('El accuracy es:', accuracy_score(y_test, xgb.predict(X_test)))



[19:42:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
El valor del AUC es:  0.9888888888888888
El accuracy es: 0.9791666666666666


C:\Users\Maty\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [86]:
importance_xgb=pd.DataFrame({'features': X.columns,'importance':xgb.feature_importances_}).sort_values(by='importance',ascending=False)

importance_xgb.head(10)


,features,importance
31,min_per_conceded_overall,0.403257
30,conceded_per_90_overall,0.157069
2,minutes_played_overall,0.145273
50,Current Club_Manchester City,0.091529
8,goals_overall,0.079715
18,clean_sheets_away,0.043778
21,conceded_away,0.021856
24,goals_involved_per_90_overall,0.020706
16,clean_sheets_overall,0.018265
12,assists_home,0.014521


In [87]:
max(df_def.clean_sheets_away)

0.0004009330806239977

In [88]:

xgb.save_model("model_def.json")